In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from joblib import load
import keras_tuner as kt
from tensorflow import keras


In [2]:
def build_nn_from_params(best_params, input_dim: int) -> keras.Model:
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))

    n_layers = best_params["n_layers"]

    for i in range(n_layers):
        units = best_params[f"units_{i}"]
        dropout_rate = best_params[f"dropout_{i}"]

        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))

    # Output layer (regresyon)
    model.add(layers.Dense(1, activation="linear"))

    lr = best_params["learning_rate"]

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=keras.losses.Huber(),
        metrics=["mae"]
    )

    return model


In [3]:
nn_best_params = {'n_layers': 2, 
           'units_0': 64,
           'dropout_0': 0.1, 
           'units_1': 64, 
           'dropout_1': 0.2, 
           'learning_rate': 0.00010278546049636793, 
           'units_2': 160, 
           'dropout_2': 0.4, 
           'units_3': 256, 
           'dropout_3': 0.1, 
           'units_4': 160, 
           'dropout_4': 0.2}

xgboost_best_params = {'max_depth': 5,
                       'learning_rate': 0.02561431711760844, 
                       'min_child_weight': 20,
                       'subsample': 0.6772757265866932,
                       'colsample_bytree': 0.9978651278689735,
                       'n_estimators': 265}


In [4]:
df = pd.read_csv("final_data.csv")

In [5]:
X = df.drop("popularity", axis=1).values
y = df["popularity"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

input_dim = X_train.shape[1]

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof_xgb = np.zeros(len(X_train))
oof_nn = np.zeros(len(X_train))

In [7]:
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train), start=1):
    print(f"Fold {fold} / 5")

    X_tr, X_val = X_train[tr_idx], X_train[val_idx]
    y_tr, y_val = y_train[tr_idx], y_train[val_idx]

    # --- XGBoost (fold modeli) ---
    xgb = XGBRegressor(
        **xgboost_best_params,
        objective="reg:squarederror",
        random_state=42
    )
    xgb.fit(X_tr, y_tr)
    oof_xgb[val_idx] = xgb.predict(X_val)

    # --- Neural Network (fold modeli) ---
    nn = build_nn_from_params(nn_best_params, input_dim=input_dim)
    nn.fit(X_tr, y_tr, epochs=200, batch_size=32, verbose=1)
    oof_nn[val_idx] = nn.predict(X_val).flatten()


Fold 1 / 5
Epoch 1/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 38.6644 - mae: 39.1560
Epoch 2/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 37.8822 - mae: 38.3728
Epoch 3/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.6365 - mae: 37.1270
Epoch 4/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 34.7606 - mae: 35.2525
Epoch 5/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 32.2333 - mae: 32.7268
Epoch 6/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 29.1406 - mae: 29.6352
Epoch 7/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 25.5975 - mae: 26.0923
Epoch 8/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 21.9777 - mae: 22.4724
Epoch 9/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 18.6891 - mae: 19.1830
Epoch 10/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 15.8853 - mae: 16.3771
Epoch 11/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.8810 - mae: 14.3713
Epoch 12/200
568/568 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1

In [8]:
meta_train = np.column_stack([oof_xgb, oof_nn])
print("meta_train shape:", meta_train.shape)
print("y_train shape   :", y_train.shape)

meta_train shape: (22684, 2)
y_train shape   : (22684,)


In [9]:
# Full-train XGBoost
final_xgb = XGBRegressor(
    max_depth=xgboost_best_params['max_depth'],
    learning_rate=xgboost_best_params['learning_rate'],
    min_child_weight=xgboost_best_params['min_child_weight'],
    subsample=xgboost_best_params['subsample'],
    colsample_bytree=xgboost_best_params['colsample_bytree'],
    n_estimators=xgboost_best_params['n_estimators'],
    objective="reg:squarederror",
    random_state=42
)
final_xgb.fit(X_train, y_train)

# Full-train NN
final_nn = build_nn_from_params(nn_best_params, input_dim=input_dim)
final_nn.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1)

# Test tahminleri
pred_xgb_test = final_xgb.predict(X_test)
pred_nn_test = final_nn.predict(X_test).flatten()

meta_test = np.column_stack([pred_xgb_test, pred_nn_test])

Epoch 1/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 38.5326 - mae: 39.0186
Epoch 2/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 37.4618 - mae: 37.9497
Epoch 3/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 35.5251 - mae: 36.0149
Epoch 4/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 32.5378 - mae: 33.0308
Epoch 5/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.6653 - mae: 29.1594  
Epoch 6/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 24.2811 - mae: 24.7757
Epoch 7/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 19.8987 - mae: 20.3930
Epoch 8/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 16.1905 - mae: 16.6830
Epoch 9/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.5892 - mae: 14.0797
Epoch 10/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.2652 - mae: 12.7547
Epoch 11/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 11.7402 - mae: 12.2282
Epoch 12/200
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 11.5230 - 

In [10]:
meta_model = Ridge(alpha=1.0)
meta_model.fit(meta_train, y_train)

final_pred = meta_model.predict(meta_test)


In [12]:
# rmse = mean_squared_error(y_test, final_pred, squared=False)
# mae = mean_absolute_error(y_test, final_pred)
# r2 = r2_score(y_test, final_pred)

print("MAE:", mean_absolute_error(y_test, final_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, final_pred)))
print("R2:", r2_score(y_test, final_pred))

# print("STACKING ENSEMBLE (XGB + NN + Ridge)")
# print("RMSE:", rmse)
# print("MAE :", mae)
# print("R²  :", r2)



MAE: 9.664645929161628
RMSE: 14.689953604803053
R2: 0.6173466731093298
